In [1]:
import requests
import json
import pandas as pd
from random import randint
import time
import timeit
import glob

datestr = time.strftime("%Y%m%d")

In [2]:
zipcodes = pd.read_csv('https://gist.githubusercontent.com/erichurst/7882666/raw/5bdc46db47d9515269ab12ed6fb2850377fd869e/US%2520Zip%2520Codes%2520from%25202013%2520Government%2520Data')
zipcodes.ZIP = [str(x).zfill(5) for x in zipcodes.ZIP]

In [4]:
def getCoords(zipcode):
    lat = zipcodes[zipcodes.ZIP==zipcode].LAT.values[0]
    lng = zipcodes[zipcodes.ZIP==zipcode].LNG.values[0]
    return [lat,lng]

In [22]:
# create function to return .json results from cargurus API
def cargurus(zipcode,dist='all',page_num=1):
    coords = getCoords(zipcode)
    options = dict(
        lat = coords[0],
        lng = coords[1],
        pagenum = page_num,
        dist = dist
        )
    
    json_url = 'https://www.cargurus.com/Cars/inventorylisting/ajaxFetchSubsetInventoryListing.action?\
    sourceContext=forSaleTab_false_0/\
    zip=&\
    address=&\
    latitude={lat}&\
    longitude={lng}&\
    distance={dist}&\
    selectedEntity=&\
    entitySelectingHelper.selectedEntity2=&\
    minPrice=&\
    maxPrice=&\
    minMileage=&\
    maxMileage=&\
    transmission=ANY&\
    bodyTypeGroup=&\
    serviceProvider=&\
    page={pagenum}&\
    filterBySourcesString=&\
    filterFeaturedBySourcesString=&\
    displayFeaturedListings=true&\
    searchSeoPageType=&\
    inventorySearchWidgetType=AUTO&\
    allYearsForTrimName=false&\
    daysOnMarketMin=&\
    daysOnMarketMax=&\
    vehicleDamageCategoriesRaw=&\
    minCo2Emission=&\
    maxCo2Emission=&\
    vatOnly=false&\
    minEngineDisplacement=&\
    maxEngineDisplacement=&\
    minMpg=&\
    maxMpg=&\
    isRecentSearchView=false'.format(**options).replace(' ','')
      
    raw = requests.get(json_url).text
    data = json.loads(raw)
    
    return data

In [179]:
def getData(min_zip='00000',max_zip='99999',pages=1,dist='all'):
    ziplist = zipcodes[(zipcodes.ZIP >= min_zip) & (zipcodes.ZIP <= max_zip)].ZIP.values
    
    df = pd.DataFrame(cargurus('10543',dist=dist,page_num=1)['listings'])
    df = df.iloc[0:0]
    for zipcode in ziplist: #zipcodes.ZIP.values:
        for i in range(0,pages):
            print("Zipcode: "+zipcode+'; Page #'+str(i+1))
            df = pd.concat([df,pd.DataFrame(cargurus(zipcode,dist=dist,page_num=1+i)['listings'])])
            # COUNT ADDITIONAL NEW CARS PER PAGE; ITERATE UNTIL NO NEW CARS
        df.drop_duplicates(inplace=True,subset=['vehicleIdentifier'])
        print(df.shape[0],'cars scraped')
    
    path = !pwd
    df.to_pickle(path[0]+'/../data/cargurus/'+datestr+'_'+min_zip+'_'+max_zip+'_'+str(pages)+'_'+str(dist)+'dist'+'.pkl')
        
    return df

In [180]:
print("SYNTAX: getData(min_zip='00000',max_zip='01000',pages=1,dist='all')")

SYNTAX: getData(min_zip='00000',max_zip='01000',pages=1,dist='all')


In [183]:
# cars = getData(min_zip='00000',max_zip='10000',pages=3,dist=10)

Zipcode: 10001; Page #1
2000
Zipcode: 10001; Page #2
4000
Zipcode: 10001; Page #3
6000
2030 cars scraped
Zipcode: 10002; Page #1
4030
Zipcode: 10002; Page #2
6030
Zipcode: 10002; Page #3
8030
2385 cars scraped
Zipcode: 10003; Page #1
4385
Zipcode: 10003; Page #2
6385
Zipcode: 10003; Page #3
8385
2385 cars scraped
Zipcode: 10004; Page #1
4385
Zipcode: 10004; Page #2
6385
Zipcode: 10004; Page #3
8385
2901 cars scraped
Zipcode: 10005; Page #1
4901
Zipcode: 10005; Page #2
6901
Zipcode: 10005; Page #3
8901
2987 cars scraped
Zipcode: 10006; Page #1
4987
Zipcode: 10006; Page #2
6987
Zipcode: 10006; Page #3
8987
2987 cars scraped
Zipcode: 10007; Page #1
4987
Zipcode: 10007; Page #2
6987
Zipcode: 10007; Page #3
8987
3001 cars scraped
Zipcode: 10009; Page #1
5001
Zipcode: 10009; Page #2
7001
Zipcode: 10009; Page #3
9001
3024 cars scraped
Zipcode: 10010; Page #1
5024
Zipcode: 10010; Page #2
7024
Zipcode: 10010; Page #3
9024
3025 cars scraped
Zipcode: 10011; Page #1
5025
Zipcode: 10011; Page #2
70

13370
7382 cars scraped
Zipcode: 10310; Page #1
9382
Zipcode: 10310; Page #2
11382
Zipcode: 10310; Page #3
13382
7382 cars scraped
Zipcode: 10311; Page #1
9382
Zipcode: 10311; Page #2
11382
Zipcode: 10311; Page #3
13382
7420 cars scraped
Zipcode: 10312; Page #1
9420
Zipcode: 10312; Page #2
11420
Zipcode: 10312; Page #3
13420
7429 cars scraped
Zipcode: 10314; Page #1
9429
Zipcode: 10314; Page #2
11429
Zipcode: 10314; Page #3
13429
7450 cars scraped
Zipcode: 10451; Page #1
9450
Zipcode: 10451; Page #2
11450
Zipcode: 10451; Page #3
13450
7474 cars scraped
Zipcode: 10452; Page #1
9474
Zipcode: 10452; Page #2
11474
Zipcode: 10452; Page #3
13474
7474 cars scraped
Zipcode: 10453; Page #1
9474
Zipcode: 10453; Page #2
11474
Zipcode: 10453; Page #3
13474
7474 cars scraped
Zipcode: 10454; Page #1
9474
Zipcode: 10454; Page #2
11474
Zipcode: 10454; Page #3
13474
7648 cars scraped
Zipcode: 10455; Page #1
9648
Zipcode: 10455; Page #2
11648
Zipcode: 10455; Page #3
13648
7774 cars scraped
Zipcode: 1045

20512
Zipcode: 10576; Page #3
21490
18599 cars scraped
Zipcode: 10577; Page #1
20599
Zipcode: 10577; Page #2
22599
Zipcode: 10577; Page #3
24599
18600 cars scraped
Zipcode: 10578; Page #1
19903
Zipcode: 10578; Page #2
21191
Zipcode: 10578; Page #3
22464
18600 cars scraped
Zipcode: 10579; Page #1
19375
Zipcode: 10579; Page #2
20135
Zipcode: 10579; Page #3
20880
18600 cars scraped
Zipcode: 10580; Page #1
20600
Zipcode: 10580; Page #2
22600
Zipcode: 10580; Page #3
24600
18687 cars scraped
Zipcode: 10583; Page #1
20687
Zipcode: 10583; Page #2
22687
Zipcode: 10583; Page #3
24687
18687 cars scraped
Zipcode: 10588; Page #1
19797
Zipcode: 10588; Page #2
20892
Zipcode: 10588; Page #3
21972
18687 cars scraped
Zipcode: 10589; Page #1
19845
Zipcode: 10589; Page #2
20988
Zipcode: 10589; Page #3
22116
18687 cars scraped
Zipcode: 10590; Page #1
19687
Zipcode: 10590; Page #2
20672
Zipcode: 10590; Page #3
21642
18687 cars scraped
Zipcode: 10591; Page #1
20687
Zipcode: 10591; Page #2
22687
Zipcode: 1059

23336
22712 cars scraped
Zipcode: 10976; Page #1
24712
Zipcode: 10976; Page #2
26712
Zipcode: 10976; Page #3
28712
22712 cars scraped
Zipcode: 10977; Page #1
24712
Zipcode: 10977; Page #2
26712
Zipcode: 10977; Page #3
28712
22712 cars scraped
Zipcode: 10979; Page #1
22876
Zipcode: 10979; Page #2
23025
Zipcode: 10979; Page #3
23159
22713 cars scraped
Zipcode: 10980; Page #1
23434
Zipcode: 10980; Page #2
24140
Zipcode: 10980; Page #3
24831
22715 cars scraped
Zipcode: 10983; Page #1
24715
Zipcode: 10983; Page #2
26715
Zipcode: 10983; Page #3
28715
22742 cars scraped
Zipcode: 10984; Page #1
24742
Zipcode: 10984; Page #2
26742
Zipcode: 10984; Page #3
28742
22795 cars scraped
Zipcode: 10985; Page #1
22924
Zipcode: 10985; Page #2
23038
Zipcode: 10985; Page #3
23137
22858 cars scraped
Zipcode: 10986; Page #1
23579
Zipcode: 10986; Page #2
24285
Zipcode: 10986; Page #3
24976
22858 cars scraped
Zipcode: 10987; Page #1
23497
Zipcode: 10987; Page #2
24121
Zipcode: 10987; Page #3
24730
22902 cars sc

27302
Zipcode: 11351; Page #2
29302
Zipcode: 11351; Page #3
31302
25302 cars scraped
Zipcode: 11354; Page #1
27302
Zipcode: 11354; Page #2
29302
Zipcode: 11354; Page #3
31302
25302 cars scraped
Zipcode: 11355; Page #1
27302
Zipcode: 11355; Page #2
29302
Zipcode: 11355; Page #3
31302
25302 cars scraped
Zipcode: 11356; Page #1
27302
Zipcode: 11356; Page #2
29302
Zipcode: 11356; Page #3
31302
25302 cars scraped
Zipcode: 11357; Page #1
27302
Zipcode: 11357; Page #2
29302
Zipcode: 11357; Page #3
31302
25302 cars scraped
Zipcode: 11358; Page #1
27302
Zipcode: 11358; Page #2
29302
Zipcode: 11358; Page #3
31302
25302 cars scraped
Zipcode: 11359; Page #1
27302
Zipcode: 11359; Page #2
29302
Zipcode: 11359; Page #3
31302
25302 cars scraped
Zipcode: 11360; Page #1
27302
Zipcode: 11360; Page #2
29302
Zipcode: 11360; Page #3
31302
25302 cars scraped
Zipcode: 11361; Page #1
27302
Zipcode: 11361; Page #2
29302
Zipcode: 11361; Page #3
31302
25302 cars scraped
Zipcode: 11362; Page #1
27302
Zipcode: 1136

...

69237
Zipcode: 14040; Page #2
69472
Zipcode: 14040; Page #3
69692
68987 cars scraped
Zipcode: 14041; Page #1
69026
Zipcode: 14041; Page #2
69050
Zipcode: 14041; Page #3
69059
68987 cars scraped
Zipcode: 14042; Page #1
69041
Zipcode: 14042; Page #2
69080
Zipcode: 14042; Page #3
69104
68988 cars scraped
Zipcode: 14043; Page #1
70988
Zipcode: 14043; Page #2
72988
Zipcode: 14043; Page #3
74988
69042 cars scraped
Zipcode: 14047; Page #1
69646
Zipcode: 14047; Page #2
70235
Zipcode: 14047; Page #3
70809
69589 cars scraped
Zipcode: 14048; Page #1
69976
Zipcode: 14048; Page #2
70348
Zipcode: 14048; Page #3
70705
69964 cars scraped
Zipcode: 14051; Page #1
71964
Zipcode: 14051; Page #2
73964
Zipcode: 14051; Page #3
75964
70010 cars scraped
Zipcode: 14052; Page #1
70571
Zipcode: 14052; Page #2
71117
Zipcode: 14052; Page #3
71648
70134 cars scraped
Zipcode: 14054; Page #1
70700
Zipcode: 14054; Page #2
71251
Zipcode: 14054; Page #3
71787
70235 cars scraped
Zipcode: 14055; Page #1
70339
Zipcode: 1405

76347
Zipcode: 14209; Page #3
78347
72347 cars scraped
Zipcode: 14210; Page #1
74347
Zipcode: 14210; Page #2
76333
Zipcode: 14210; Page #3
78304
72347 cars scraped
Zipcode: 14211; Page #1
74347
Zipcode: 14211; Page #2
76347
Zipcode: 14211; Page #3
78347
72347 cars scraped
Zipcode: 14212; Page #1
74347
Zipcode: 14212; Page #2
76347
Zipcode: 14212; Page #3
78347
72347 cars scraped
Zipcode: 14213; Page #1
74347
Zipcode: 14213; Page #2
76347
Zipcode: 14213; Page #3
78347
72347 cars scraped
Zipcode: 14214; Page #1
74347
Zipcode: 14214; Page #2
76347
Zipcode: 14214; Page #3
78347
72347 cars scraped
Zipcode: 14215; Page #1
74347
Zipcode: 14215; Page #2
76347
Zipcode: 14215; Page #3
78347
72347 cars scraped
Zipcode: 14216; Page #1
74347
Zipcode: 14216; Page #2
76347
Zipcode: 14216; Page #3
78347
72347 cars scraped
Zipcode: 14217; Page #1
74347
Zipcode: 14217; Page #2
76347
Zipcode: 14217; Page #3
78347
72347 cars scraped
Zipcode: 14218; Page #1
74347
Zipcode: 14218; Page #2
76347
Zipcode: 1421

80374
80374 cars scraped
Zipcode: 14513; Page #1
80440
Zipcode: 14513; Page #2
80491
Zipcode: 14513; Page #3
80527
80374 cars scraped
Zipcode: 14514; Page #1
82374
Zipcode: 14514; Page #2
84374
Zipcode: 14514; Page #3
86374
80374 cars scraped
Zipcode: 14516; Page #1
80639
Zipcode: 14516; Page #2
80889
Zipcode: 14516; Page #3
81124
80374 cars scraped
Zipcode: 14517; Page #1
80447
Zipcode: 14517; Page #2
80505
Zipcode: 14517; Page #3
80548
80374 cars scraped
Zipcode: 14519; Page #1
81467
Zipcode: 14519; Page #2
82545
Zipcode: 14519; Page #3
83608
80464 cars scraped
Zipcode: 14521; Page #1
80464
Zipcode: 14521; Page #2
80464
Zipcode: 14521; Page #3
80464
80464 cars scraped
Zipcode: 14522; Page #1
81132
Zipcode: 14522; Page #2
81785
Zipcode: 14522; Page #3
82423
80464 cars scraped
Zipcode: 14525; Page #1
80574
Zipcode: 14525; Page #2
80669
Zipcode: 14525; Page #3
80749
80464 cars scraped
Zipcode: 14526; Page #1
82464
Zipcode: 14526; Page #2
84464
Zipcode: 14526; Page #3
86464
80534 cars sc

81534
Zipcode: 14711; Page #2
81534
Zipcode: 14711; Page #3
81534
81533 cars scraped
Zipcode: 14712; Page #1
81941
Zipcode: 14712; Page #2
82334
Zipcode: 14712; Page #3
82712
81533 cars scraped
Zipcode: 14714; Page #1
81534
Zipcode: 14714; Page #2
81534
Zipcode: 14714; Page #3
81534
81533 cars scraped
Zipcode: 14715; Page #1
81533
Zipcode: 14715; Page #2
81533
Zipcode: 14715; Page #3
81533
81533 cars scraped
Zipcode: 14716; Page #1
81989
Zipcode: 14716; Page #2
82430
Zipcode: 14716; Page #3
82856
81614 cars scraped
Zipcode: 14717; Page #1
81615
Zipcode: 14717; Page #2
81615
Zipcode: 14717; Page #3
81615
81614 cars scraped
Zipcode: 14718; Page #1
81980
Zipcode: 14718; Page #2
82331
Zipcode: 14718; Page #3
82667
81614 cars scraped
Zipcode: 14719; Page #1
81631
Zipcode: 14719; Page #2
81633
Zipcode: 14719; Page #3
81633
81614 cars scraped
Zipcode: 14720; Page #1
82023
Zipcode: 14720; Page #2
82417
Zipcode: 14720; Page #3
82796
81614 cars scraped
Zipcode: 14721; Page #1
81912
Zipcode: 1472

82804
Zipcode: 14822; Page #3
82815
82737 cars scraped
Zipcode: 14823; Page #1
82766
Zipcode: 14823; Page #2
82780
Zipcode: 14823; Page #3
82780
82737 cars scraped
Zipcode: 14824; Page #1
82740
Zipcode: 14824; Page #2
82740
Zipcode: 14824; Page #3
82740
82737 cars scraped
Zipcode: 14825; Page #1
82911
Zipcode: 14825; Page #2
83070
Zipcode: 14825; Page #3
83214
82737 cars scraped
Zipcode: 14826; Page #1
82777
Zipcode: 14826; Page #2
82802
Zipcode: 14826; Page #3
82812
82737 cars scraped
Zipcode: 14827; Page #1
82875
Zipcode: 14827; Page #2
82998
Zipcode: 14827; Page #3
83106
82737 cars scraped
Zipcode: 14830; Page #1
82951
Zipcode: 14830; Page #2
83150
Zipcode: 14830; Page #3
83334
82737 cars scraped
Zipcode: 14836; Page #1
82737
Zipcode: 14836; Page #2
82737
Zipcode: 14836; Page #3
82737
82737 cars scraped
Zipcode: 14837; Page #1
82739
Zipcode: 14837; Page #2
82739
Zipcode: 14837; Page #3
82739
82737 cars scraped
Zipcode: 14838; Page #1
82941
Zipcode: 14838; Page #2
83130
Zipcode: 1483